In [1]:
# TODO which one?
#git clone https://github.com/lucidrains/iTransformer.git
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer
import torch
from pathlib import Path

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from utils import data_handling, helpers, training_functions
import config
import pandas as pd

window_size = 96
pred_length = (96)

four_weeks = -24*7*4
print("Import succesfull")

Import succesfull


# Select dataset for transfer learning

In [2]:
# electricity dataset
data_dict = data_handling.load_electricity()

electricity = {}
electricity["dataloader_train"], electricity["dataloader_validation"], electricity["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)

# create a smaller subset of the train dataset
electricity["4_weeks_train"] = data_dict["train"][four_weeks:,:]
electricity["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(electricity["4_weeks_train"] , window_size, pred_length)
electricity["4_weeks_train"] = data_handling.DataLoader(electricity["4_weeks_train"] , batch_size=32, shuffle=True)



# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to datalaoder
bavaria = {}
bavaria["dataloader_train"], bavaria["dataloader_validation"], bavaria["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)

# add fine-tuning datalaoders
# create a smaller subset of the train dataset
bavaria["4_weeks_train"] = data_dict["train"][four_weeks:,:]
bavaria["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(bavaria["4_weeks_train"] , window_size, pred_length)
bavaria["4_weeks_train"] = data_handling.DataLoader(bavaria["4_weeks_train"] , batch_size=32, shuffle=True)



# building genome project dataset
data_tensor = data_handling.load_genome_project_data()
gp_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
# convert to datalaoder
genome_project = {}
genome_project["dataloader_train"], genome_project["dataloader_validation"], genome_project["dataloader_test"] = data_handling.convert_data(gp_dict, window_size, pred_length)

# add fine-tuning datalaoders
# create a smaller subset of the train dataset
genome_project["4_weeks_train"] = gp_dict["train"][four_weeks:,:]
genome_project["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(genome_project["4_weeks_train"] , window_size, pred_length)
genome_project["4_weeks_train"] = data_handling.DataLoader(genome_project["4_weeks_train"] , batch_size=32, shuffle=True)



# merge in dataset dict
datasets = {"electricity" : electricity,
            "genome_project" : genome_project,
			"bavaria" : bavaria
            }


# define parameters for all models
best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.2, 
                        'num_mem_tokens': 4, 'learning_rate': 0.0005}

Feature batch shape: torch.Size([32, 96, 348])


Feature batch shape: torch.Size([32, 96, 59])
Feature batch shape: torch.Size([32, 96, 1454])


# Zero-shot prediction on the test sets

In [3]:
zero_shot_metrics = pd.DataFrame()

for source, value in datasets.items():
    model_path = config.CONFIG_MODEL_LOCATION["iTransformer_baseline"] / source / f"{source}_full_dataset_best_val_loss.pt"

    # select the 2 target datasets
    target_keys = [data_key for data_key in datasets if data_key != source]

    # load model, fit to target dataset and predict
    for target in target_keys:
        dataloader = datasets[target]["dataloader_test"]

        inputs, _ = next(iter(dataloader))
        num_variates = inputs.size(2)

        model_config = {
            'num_variates': num_variates,
            'lookback_len': window_size,
            'depth': best_parameters["depth"],
            'dim': best_parameters["dim"],
            'num_tokens_per_variate': 1,
            'pred_length': pred_length,
            'dim_head': best_parameters["dim_head"],
            'heads': best_parameters["heads"],
            'attn_dropout': best_parameters["attn_dropout"],
            'ff_mult': best_parameters["ff_mult"],
            'ff_dropout': best_parameters["ff_dropout"],
            'num_mem_tokens': best_parameters["num_mem_tokens"],
            'use_reversible_instance_norm': True,
            'reversible_instance_norm_affine': False,
            'flash_attn': True
        }

        device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")


        checkpoint = torch.load(model_path,map_location='cpu')

        # values are all set to zero (beta) and one (gamma), array needs to be adapted to num_variates
        # learned affine parameters are series specific and need to be relearned for new series
        # value are kept at 1 and 0 for stationary normalization
        checkpoint["model_state_dict"]["reversible_instance_norm.beta"] = torch.zeros(num_variates, 1, dtype=torch.float)
        checkpoint["model_state_dict"]["reversible_instance_norm.gamma"] = torch.ones(num_variates, 1, dtype=torch.float)


        model = iTransformer(**model_config).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])


        metrics_dict = helpers.full_eval(model, dataloader, device)
        metrics = metrics_dict[96]["mse"].item(),  metrics_dict[96]["mae"].item()
        
        zero_shot_metrics[f"{source}_to_{target}"] = pd.Series(metrics)


zero_shot_metrics.index = ["mse_zero_shot", "mae_zero_shot"]
zero_shot_metrics


Using device: cuda:1
Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


Epoch: Validating: 100%|██████████| 86/86 [00:18<00:00,  4.70it/s]


Using device: cuda:1


Epoch: Validating: 100%|██████████| 83/83 [00:00<00:00, 236.89it/s]


Using device: cuda:1


...

Using device: cuda:1


Epoch: Validating: 100%|██████████| 86/86 [00:10<00:00,  7.88it/s]


Using device: cuda:1


Epoch: Validating: 100%|██████████| 86/86 [00:12<00:00,  7.11it/s]


,electricity_to_genome_project,electricity_to_bavaria,genome_project_to_electricity,genome_project_to_bavaria,bavaria_to_electricity,bavaria_to_genome_project
mse_zero_shot,0.610017,0.219871,0.363649,0.174696,1.971902,1.959411
mae_zero_shot,0.448888,0.443461,0.411646,0.384142,1.124707,0.969583


# Fine-tuning and predicitons on the test data

We fine tune for 5 epochs on different target datasets training sets length. After every epoch the training and validation loss is logged. For the final evaluation the model with the best validation loss is selected. 

In [5]:
def fine_tune(model, dataloader_train, dataloader_validation, device, epoch=1):

    # defining all needed instances
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["iTransformer_baseline"])

    # run model training as mentioned in the original paper
    metrics, model = training_functions.train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, scheduler, writer, save_model=False)
    return model, device




fine_tuning_metrics_df = pd.DataFrame()
epoch=5

# select a source dataset
for source, value in datasets.items():
    print(f"Using {source} as a source dataset.")
    model_path = config.CONFIG_MODEL_LOCATION["iTransformer_baseline"] / source / f"{source}_full_dataset_best_val_loss.pt"

    # select both target datasets
    target_keys = [data_key for data_key in datasets if data_key != source]

    # do test predictions for both target datasets
    for target in target_keys:
        fine_tuning_metrics_list = []
        dataloader_test = datasets[target]["dataloader_test"]
        datalaoder_val = datasets[target]["dataloader_validation"]

        inputs, _ = next(iter(dataloader_test))
        num_variates = inputs.size(2)

        model_config = {
            'num_variates': num_variates,
            'lookback_len': window_size,
            'depth': best_parameters["depth"],
            'dim': best_parameters["dim"],
            'num_tokens_per_variate': 1,
            'pred_length': pred_length,
            'dim_head': best_parameters["dim_head"],
            'heads': best_parameters["heads"],
            'attn_dropout': best_parameters["attn_dropout"],
            'ff_mult': best_parameters["ff_mult"],
            'ff_dropout': best_parameters["ff_dropout"],
            'num_mem_tokens': best_parameters["num_mem_tokens"],
            'use_reversible_instance_norm': True,
            'reversible_instance_norm_affine': True,
            'flash_attn': True
        }

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")


        checkpoint = torch.load(model_path,map_location='cpu')

        # values are all set to zero (beta) and one (gamma), array needs to be adapted to num_variates
        # learned affine parameters are series specific and need to be relearned for new series
        # value are kept at 1 and 0 for stationary normalization
        checkpoint["model_state_dict"]["reversible_instance_norm.beta"] = torch.zeros(num_variates, 1, dtype=torch.float)
        checkpoint["model_state_dict"]["reversible_instance_norm.gamma"] = torch.ones(num_variates, 1, dtype=torch.float)


        model = iTransformer(**model_config).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])

        
        # fne tuning on 4 weeks of target data
        fine_tune_dataloader = datasets[target]["4_weeks_train"]        
        model, device = fine_tune(model, fine_tune_dataloader, datalaoder_val, device, epoch=epoch)


        metrics_fine_tuned = helpers.full_eval(model, dataloader_test, device)
        fine_tuning_metrics_list.append(metrics_fine_tuned[96]["mse"].item())
        fine_tuning_metrics_list.append(metrics_fine_tuned[96]["mae"].item())


        # train on full target dataset
        checkpoint = torch.load(model_path,map_location='cpu')

        # values are all set to zero (beta) and one (gamma), array needs to be adapted to num_variates
        # learned affine parameters are series specific and need to be relearned for new series
        # value are kept at 1 and 0 for stationary normalization
        checkpoint["model_state_dict"]["reversible_instance_norm.beta"] = torch.zeros(num_variates, 1, dtype=torch.float)
        checkpoint["model_state_dict"]["reversible_instance_norm.gamma"] = torch.ones(num_variates, 1, dtype=torch.float)

        model = iTransformer(**model_config).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])

        # fne tuning on 4 weeks of target data
        fine_tune_dataloader = datasets[target]["dataloader_train"]        
        model, device = fine_tune(model, fine_tune_dataloader, datalaoder_val, device, epoch=epoch)

        metrics_fine_tuned = helpers.full_eval(model, dataloader_test, device)
        fine_tuning_metrics_list.append(metrics_fine_tuned[96]["mse"].item())
        fine_tuning_metrics_list.append(metrics_fine_tuned[96]["mae"].item())


        fine_tuning_metrics_df[f"{source}_to_{target}"] = pd.Series(fine_tuning_metrics_list)



fine_tuning_metrics_df.index = ["mse_4_weeks_tuning", "mae_4_weeks_tuning", "mse_full_tuning", "mae_full_tuning"]
fine_tuning_metrics_df

Using device: cuda:0


Epoch: Validating: 100%|██████████| 40/40 [00:07<00:00,  5.23it/s]


Validation metrics: {'mse': tensor(0.3709, device='cuda:0')}


Epoch: Validating: 100%|██████████| 40/40 [00:05<00:00,  7.29it/s]


Validation metrics: {'mse': tensor(0.3638, device='cuda:0')}


...

Validation metrics: {'mse': tensor(0.3234, device='cuda:0')}


Epoch: Validating: 100%|██████████| 40/40 [00:19<00:00,  2.04it/s]


Validation metrics: {'mse': tensor(0.3271, device='cuda:0')}


Epoch: Validating: 100%|██████████| 86/86 [00:47<00:00,  1.83it/s]


,electricity_to_genome_project,electricity_to_bavaria,genome_project_to_electricity,genome_project_to_bavaria,bavaria_to_electricity,bavaria_to_genome_project
mse_4_weeks_tuning,0.521376,0.001411,0.221595,0.000589,0.322062,0.576865
mae_4_weeks_tuning,0.387570,0.028127,0.303785,0.017151,0.394773,0.441728
mse_full_tuning,0.464582,0.000374,0.180445,0.002362,0.189756,0.461987
mae_full_tuning,0.361495,0.013917,0.265282,0.031661,0.278883,0.366178


In [6]:
result = pd.concat([zero_shot_metrics, fine_tuning_metrics_df], ignore_index=False)

output_path = config.CONFIG_OUTPUT_PATH["iTransformer_baseline"] / "itransformer_full_tl_metrics.csv"

result.to_csv(output_path)